In [14]:
import random
import numpy as np 
import math
import time 
# import multiprocessing
import concurrent.futures

1. Основы программирования
2. Основы BigData
3. Основы баз данных

### 1. Основы программирования

***1. Дан массив целых чисел nums и целое число target. Написать функцию, возвращающую индексы элементов, дающих в сумме число target?***

In [15]:
def func(nums, target, toggle=True):
    ''' nums - массив целых чисел (list or tuple)
        target - integer
        toggle - если True, то складывать число с самим собой нельзя
    '''
    if not isinstance(lst, (list, tuple)):
        raise TypeError( 'Должен быть массив целых чисел (list or tuple)') 
        
    idx = []
    for i in range(len(nums)):
        for j in range(len(nums)):
            
            if i == j and toggle:
                continue 
            
            if nums[i]+nums[j] == target:
                
                if (j, i) in idx:
                    continue
                else:
                    idx.append((i, j))
    if idx:        
        return idx
    else:
        txt = 'Среди данного массива нет чисел отвечающих условиям задачи'
        return txt

# генерируем массив из случайных чисел
# random.seed(33)
rng = range(random.randint(1, 10))
lst = [random.randint(1, 10) for i in rng]
print('Массив: ', lst)

# генерируем случайное число - target
# random.seed(33)
target = random.randint(1, 15)
print('target:', target)

res = func(lst, target, False)
print('Искомые пары индексов:', res)

Массив:  [2, 6, 5, 4, 2, 2, 9, 7, 6, 4]
target: 2
Искомые пары индексов: Среди данного массива нет чисел отвечающих условиям задачи


***2. Предполагается, что в каждом массиве имеется не больше одной пары дающих в сумме заданное число target. Нельзя использовать один и тот же элемент дважды?***

Такой элемент можно использовать дважды в силу коммутативности операции сложения целых чисел (множество целых чисел - коммутативное кольцо).

#### 2. Основы BigData

***3. Напиши псевдокодом последовательность расчета word count в парадигме map-reduce***

// - комментарии
```
Начало
Ввод(коллекция из слов);
    //----------MAP------------------
    множество := разбить коллекцию на n-подмножеств; 
    // где "множество" есть множество подмножеств из слов
    // всего n подмножеств (элементов множества)
    словарь := создать словарь результатов всех подсчетов;
    для каждого элемента из множества:
        результат := посчитать частоту слов;
        добавить результат в словарь;
    передать словарь для обработки в процедуру REDUCE    
    // --------REDUCE-----------------
    общий результат := объеденить все результаты из словаря
Конец
```

###### Эмулирование MapReduce

In [16]:
nwords = 10  # 20000
np.random.seed(32)
idx = np.random.randint(nwords, size=nwords*10)
words = list(map(lambda i: f'word{i}', idx))
words = np.array(words)[idx].tolist()
print(words)

['word3', 'word7', 'word9', 'word5', 'word8', 'word3', 'word9', 'word8', 'word7', 'word9', 'word3', 'word5', 'word8', 'word5', 'word6', 'word8', 'word5', 'word6', 'word3', 'word6', 'word3', 'word3', 'word9', 'word7', 'word9', 'word9', 'word5', 'word3', 'word7', 'word9', 'word3', 'word9', 'word7', 'word8', 'word6', 'word7', 'word8', 'word6', 'word5', 'word8', 'word6', 'word6', 'word9', 'word5', 'word3', 'word7', 'word3', 'word6', 'word5', 'word6', 'word7', 'word6', 'word9', 'word9', 'word9', 'word8', 'word9', 'word7', 'word7', 'word9', 'word5', 'word7', 'word8', 'word5', 'word3', 'word5', 'word9', 'word9', 'word6', 'word9', 'word5', 'word5', 'word3', 'word5', 'word9', 'word9', 'word6', 'word5', 'word3', 'word6', 'word7', 'word3', 'word7', 'word3', 'word7', 'word7', 'word8', 'word6', 'word3', 'word5', 'word6', 'word7', 'word5', 'word9', 'word6', 'word5', 'word7', 'word8', 'word9', 'word7']


In [17]:
def split(words, nchunks):
    '''split data into chunks'''
    step = math.ceil(len(words)/nchunks)
    for i in range(0, len(words), step):
        yield words[i:i+step]

def worker_counter(chunk, numb):
    '''
        worker that count words in a collection
        chunk: a collection
        numb: a worker number
    '''
    dct = {}  # dictionary to store word counters
    print('Starting process by worker {}'.format(numb))
    
    while chunk:
        word = chunk.pop()        
        if word in dct.keys():
            dct[word] += 1 
        else:
            dct [word] = 1 
    # time.sleep(1)            
    print(f'Process is finished by worker {numb}')
    return dct

***MAP***

In [18]:
def map1(words, nchunks=2):

    processes = []  # list to store current workers
    results = [] # list to store all results of workers
    splitted_data = split(words, nchunks) # generator function

    # start time counting
    start_time = time.perf_counter()
    
    # multithread for each worker
    with concurrent.futures.ProcessPoolExecutor() as executor:
        for numb, chunk in enumerate(splitted_data, start=1):
            proc = executor.submit(worker_counter, chunk, numb)
            processes.append(proc)
    
    # extract and store result of each worker to results
    for f in concurrent.futures.as_completed(processes):
        results.append(f.result())
    
    end_time = time.perf_counter()        
    print(f'Finished  in {round(end_time - start_time, 4)} second(s)')            

    return results

print('*****mapping: splitting and processing***')
results = map1(words, 2)

*****mapping: splitting and processing***
Starting process by worker 2Starting process by worker 1

Process is finished by worker 2Process is finished by worker 1

Finished  in 0.0991 second(s)


***REDUCE***

In [19]:
def reduce(results):
    
    dct = {}  # final result
    for worker in results:        
        for key in worker.keys():
            
            if key in dct.keys():
                dct[key] += worker[key]
            else:
                dct[key] = worker[key]
    
    return dct

print('****reducing: combining results of mapping******')
res = reduce(results)
res = dict(sorted(res.items(), key=lambda x: x[1], reverse=True))
print(res)

****reducing: combining results of mapping******
{'word9': 21, 'word7': 18, 'word5': 18, 'word6': 16, 'word3': 16, 'word8': 11}


### 3. Основы баз данных

***3. Перечисли способы получить количество записей в таблице?***

Способ 1
```
SELECT COUNT(*) FROM Income
UNION
SELECT COUNT(*) FROM Outcome;
```
Способ 2
```
SELECT 
    TABLE_NAME, TABLE_ROWS
FROM
    INFORMATION_SCHEMA.TABLES
WHERE TABLE_SCHEMA = 'inc_out' AND TABLE_NAME = 'Income'
```
Способ 3
```
SHOW TABLE STATUS;
```
Способ 4
```
SELECT * FROM Income i
SELECT FOUND_ROWS(); 
```

***2. Выполнится ли этот запрос?***
```
SELECT 
     order_id,
     order_code,
     SUM(order_value)
FROM 
     orders
GROUP BY
     order_id
```
Данный запрос не выполнится поскольку столбец ***order_code*** не включен в <span style="color:green">***GROUP BY***</span> и не инкапсулирован в агрегатную функцию.